In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round
from pyspark.sql.window import Window, WindowSpec

In [4]:
spark = SparkSession.builder.master("local[*]").appName("Basics 1").getOrCreate()

In [51]:
data = [1, 2, 3, 4, 5]

In [50]:
rdd = spark.sparkContext.parallelize([data])

In [52]:
rdd.collect()

[[1, 2, 3, 4, 5]]

In [3]:
spark

In [4]:
from pyspark.sql.functions import col, when, count, lag, avg, stddev
from pyspark.sql import functions as F

In [5]:
data = [("Dinkar", 24), ("Mandeep", 23), ("Raj", 22)]
columns = ["name", "age"]

In [6]:
data1 = [{"Name":"Dinkar", "Disg":"Data Scientist"},{"Name":"Nissarg", "Disg":"Teaching assistent"},{"Name":"Mandeep", "Disg":"Data Scientist"}]

In [1]:
df1 = spark.read.csv("./sales.csv", header=True, inferSchema=True)

NameError: name 'spark' is not defined

In [42]:
rdd = df1.rdd

In [8]:
df1.dtypes

[('order_id', 'int'),
 ('order_date', 'date'),
 ('customer_id', 'string'),
 ('gender', 'string'),
 ('age', 'int'),
 ('city', 'string'),
 ('product', 'string'),
 ('category', 'string'),
 ('quantity', 'int'),
 ('price', 'int'),
 ('discount', 'int'),
 ('payment_method', 'string')]

In [9]:
df1.createOrReplaceTempView("mytable")

In [10]:
spark.sql("""
    SELECT *
    FROM mytable
    WHERE age > 20
""").show()

+--------+----------+-----------+------+---+---------+----------+--------------+--------+-----+--------+--------------+
|order_id|order_date|customer_id|gender|age|     city|   product|      category|quantity|price|discount|payment_method|
+--------+----------+-----------+------+---+---------+----------+--------------+--------+-----+--------+--------------+
|    1001|2024-03-29|       C072|     M| 38|  Kolkata|       Fan|Home Appliance|       3| 1160|       7|    Debit Card|
|    1003|2025-07-02|       C161|     F| 39|Ahmedabad|     Jeans|       Apparel|       4| 5611|      12|   Credit Card|
|    1004|2025-02-04|       C271|     F| 64|Bengaluru|      Iron|Home Appliance|    NULL|18731|      10|          Cash|
|    1005|2025-02-01|       C135|     M| 26|  Kolkata|       Fan|Home Appliance|       1|31851|       7|    Debit Card|
|    1006|2023-11-29|       C053|     F| 37|  Kolkata|     Shoes|      Footwear|       2|74269|       0|          Cash|
|    1007|2025-04-30|       C002|     F|

In [11]:
df1.select("gender", "age").show()

+------+---+
|gender|age|
+------+---+
|     M| 38|
|  NULL| 19|
|     F| 39|
|     F| 64|
|     M| 26|
|     F| 37|
|     F| 23|
|     F| 51|
|     F| 43|
|     M| 24|
|     F| 52|
|     F| 24|
|     F| 57|
|     M| 43|
|     F| 56|
|     M| 46|
|     F| 59|
|     F| 45|
|     M| 58|
|     F| 41|
+------+---+
only showing top 20 rows


In [12]:
nulls = df1.select(count(when(col("city").isNull(), "city"))).show()

+---------------------------------------------+
|count(CASE WHEN (city IS NULL) THEN city END)|
+---------------------------------------------+
|                                            4|
+---------------------------------------------+



In [13]:
# df1.join(df, on="id", how="inner")u7

In [14]:
df1.na.drop().show()

+--------+----------+-----------+------+---+---------+----------+--------------+--------+-----+--------+--------------+
|order_id|order_date|customer_id|gender|age|     city|   product|      category|quantity|price|discount|payment_method|
+--------+----------+-----------+------+---+---------+----------+--------------+--------+-----+--------+--------------+
|    1001|2024-03-29|       C072|     M| 38|  Kolkata|       Fan|Home Appliance|       3| 1160|       7|    Debit Card|
|    1003|2025-07-02|       C161|     F| 39|Ahmedabad|     Jeans|       Apparel|       4| 5611|      12|   Credit Card|
|    1005|2025-02-01|       C135|     M| 26|  Kolkata|       Fan|Home Appliance|       1|31851|       7|    Debit Card|
|    1006|2023-11-29|       C053|     F| 37|  Kolkata|     Shoes|      Footwear|       2|74269|       0|          Cash|
|    1007|2025-04-30|       C002|     F| 23|   Jaipur|Smartwatch|   Electronics|       4|36220|       5|    Debit Card|
|    1008|2024-01-17|       C218|     F|

In [15]:
nulls = df1.select([count(when(col(c).isNull(), c)).alias(c) for c in df1.columns]).toPandas()
nulls

,order_id,order_date,customer_id,gender,age,city,product,category,quantity,price,discount,payment_method
0,0,0,0,1,0,4,0,0,3,0,3,0


In [16]:
# df = spark.createDataFrame(data, columns)
# df.limit(20).collect()

In [17]:
# df = spark.createDataFrame(data1)
# df.limit(20).collect()

In [18]:
a = [1,2,3]
b = ["one","two","three"]

c = zip(a,b)

In [19]:
df1 = df1.withColumn(
    "discount_amount",
    round(col("price") * (col("discount") / 100), 2)
)

In [20]:
df1.show()

+--------+----------+-----------+------+---+---------+----------+--------------+--------+-----+--------+--------------+---------------+
|order_id|order_date|customer_id|gender|age|     city|   product|      category|quantity|price|discount|payment_method|discount_amount|
+--------+----------+-----------+------+---+---------+----------+--------------+--------+-----+--------+--------------+---------------+
|    1001|2024-03-29|       C072|     M| 38|  Kolkata|       Fan|Home Appliance|       3| 1160|       7|    Debit Card|           81.2|
|    1002|2025-01-12|       C150|  NULL| 19|Hyderabad|      Iron|Home Appliance|       4|44431|       7|          Cash|        3110.17|
|    1003|2025-07-02|       C161|     F| 39|Ahmedabad|     Jeans|       Apparel|       4| 5611|      12|   Credit Card|         673.32|
|    1004|2025-02-04|       C271|     F| 64|Bengaluru|      Iron|Home Appliance|    NULL|18731|      10|          Cash|         1873.1|
|    1005|2025-02-01|       C135|     M| 26|  Ko

In [22]:
win = Window.partitionBy("customer_id").orderBy("order_date")

In [31]:
df1.withColumn(
    "rank",
    F.rank().over(win.orderBy(F.col("price")))
).show(5643)

+--------+----------+-----------+------+---+---------+----------+--------------+--------+-----+--------+--------------+---------------+----+
|order_id|order_date|customer_id|gender|age|     city|   product|      category|quantity|price|discount|payment_method|discount_amount|rank|
+--------+----------+-----------+------+---+---------+----------+--------------+--------+-----+--------+--------------+---------------+----+
|    1638|2024-06-18|       C001|     F| 60|Hyderabad|   Blender|Home Appliance|       2|20180|       0|   Credit Card|            0.0|   1|
|    1065|2025-10-04|       C001|     M| 30|Ahmedabad|     Jeans|       Apparel|       4|30655|      12|          Cash|         3678.6|   2|
|    1830|2024-12-09|       C001|     F| 52|   Jaipur|   Sandals|      Footwear|       4|65440|      10|          Cash|         6544.0|   3|
|    1028|2025-08-23|       C002|     M| 29|  Chennai|   Sandals|      Footwear|    NULL|16671|       0|           EMI|            0.0|   1|
|    1007|202

In [29]:
df1.withColumn(
    "row_num",
    F.row_number().over(win)
).show()

+--------+----------+-----------+------+---+---------+----------+--------------+--------+-----+--------+--------------+---------------+-------+
|order_id|order_date|customer_id|gender|age|     city|   product|      category|quantity|price|discount|payment_method|discount_amount|row_num|
+--------+----------+-----------+------+---+---------+----------+--------------+--------+-----+--------+--------------+---------------+-------+
|    1638|2024-06-18|       C001|     F| 60|Hyderabad|   Blender|Home Appliance|       2|20180|       0|   Credit Card|            0.0|      1|
|    1830|2024-12-09|       C001|     F| 52|   Jaipur|   Sandals|      Footwear|       4|65440|      10|          Cash|         6544.0|      2|
|    1065|2025-10-04|       C001|     M| 30|Ahmedabad|     Jeans|       Apparel|       4|30655|      12|          Cash|         3678.6|      3|
|    1007|2025-04-30|       C002|     F| 23|   Jaipur|Smartwatch|   Electronics|       4|36220|       5|    Debit Card|         1811.0| 

In [ ]:
rdd = spark